In [1]:
from lib.load_data import *

start_year = 15
end_year = 17
truncate_size = 10000
nb_flow = 1
image_size = 20
len_test = 7 * 48
## load data
data_all = []
timestamps_all = []
for year in range(start_year, end_year):
    filepath = os.path.join('datasets', 'TaxiBJ', 'BJ{}_M32x32_T30_InOut.h5'.format(year))
    data, timestamps = load_stdata(filepath)

    data, timestamps = remove_incomplete_days(data[:truncate_size], timestamps[:truncate_size], T)

    data = data[:, :nb_flow, :image_size, :image_size]

    data[data < 0] = 0
    data_all.append(data)
    timestamps_all.extend(timestamps)
    print("year:{}, data shape: {}".format(year, data.shape))

data_train = np.vstack(copy(data_all))[:-len_test]
# [len, nb_flow, image_size, image_size]

year:15, data shape: (5520, 1, 20, 20)
year:16, data shape: (6624, 1, 20, 20)


In [2]:
timestamps_train = timestamps_all[:-len_test]

In [3]:
T = 48
weekly_timestamps = {1:[],2:[],3:[],4:[],5:[],6:[],7:[]}


In [4]:
t = timestamp2vec(timestamps_train)

In [5]:
for i in range(0, len(timestamps_train), T):
    vec = t[i]
    for j in range(7):
        if vec[j] == 1:
            weekly_timestamps[j+1].append(i)
            break

In [6]:
weekly_demand = dict()
for key in weekly_timestamps.keys():
    data = []
    for index in weekly_timestamps[key]:
        data.append(data_train[index:index+T])
    weekly_demand[key] = np.array(data).mean(axis=0)

In [7]:
weekly_demand[1].shape

(48, 1, 20, 20)

In [8]:
patterns = []
for key in weekly_demand.keys():
    patterns.append(weekly_demand[key].reshape(-1,400))

In [9]:
patterns = np.array(patterns).reshape(-1,400)

In [10]:
patterns.shape

(336, 400)

In [11]:
patterns = np.transpose(patterns, (1,0))
patterns.shape

(400, 336)

In [13]:
## every row min-max normallization
arr_mmn = np.zeros((patterns.shape[0], patterns.shape[1]))
i = 0
for data in patterns:
    data_mmn = (data - data.min()) / (data.max() - data.min())
    arr_mmn[i, :] = data_mmn
    i += 1

In [14]:
import h5py
f = h5py.File('data/weekly_pattern_mmn.h5', 'w')
f.create_dataset('weekly_pattern', data = arr_mmn)
f.close()
